In [20]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions # Importa a classe Firefox e FirefoxOptions do módulo selenium.webdriver para controlar o navegador Firefox
from selenium.common.exceptions import NoSuchElementException # Importa a exceção NoSuchElementException para tratar casos onde um elemento não é encontrado na página
from selenium.webdriver.common.by import By # Importa By para facilitar a localização de elementos na página usando diferentes estratégias (ID, XPATH, CSS_SELECTOR, etc.)
from unidecode import unidecode # Importa a função unidecode para converter caracteres Unicode em seus equivalentes ASCII, útil para normalização de texto
import pandas as pd # Importa a biblioteca pandas, que é uma poderosa ferramenta para análise e manipulação de dados
import numpy as np #Importa a biblioteca numpy, que fornece suporte para arrays grandes e multidimensionais, além de funções matemáticas de alto nível

In [21]:
def get_betway_odds():
    # Configuração das opções do navegador Firefox para rodar em modo headless (sem interface gráfica)
    opts = FirefoxOptions()
    opts.headless = True
    browser = webdriver.Firefox(options=opts)
    
    # Definir tempo de espera implícito de 2 segundos para carregar os elementos da página
    browser.implicitly_wait(2)

    # Acessar a página específica do site Betway para o Campeonato Brasileiro Série A
    browser.get('https://betway.com/pt/sports/grp/soccer/brazil/brasileiro-serie-a')    
    
    # Base XPath para localizar os elementos de interesse na página
    base_xpath = '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[3]/div[2]/div'

    # Listas para armazenar os dados extraídos
    games_ids = []
    mandantes = []
    visitantes = []
    odds_mandantes = []
    odds_empates = []
    odds_visitantes = []

    # Loop para percorrer dias e jogos disponíveis na página
    for day in range(1, 5):  # Verifica os primeiros 4 dias
        for game in range(1, 10):  # Verifica os primeiros 9 jogos de cada dia
            try:
                # Tentar encontrar o elemento do jogo específico usando XPath dinâmico
                text_game = browser.find_element(
                    by='xpath',
                    value=base_xpath + f'/div[{day}]/div[2]/div/div[{game}]'
                ).text

                # Separar o texto do jogo em partes específicas
                try:
                    _, game_name, odd_mandante, odd_empate, odd_visitante = text_game.split('\n')
                except ValueError:
                    continue
                
                # Processar os nomes dos clubes
                clubs = [unidecode(club.strip()).lower().replace(' ', '_') for club in game_name.split('-')]
                game_id = '-'.join(sorted(clubs))  # Criar um ID único para o jogo
                mandante = clubs[0]
                visitante = clubs[1]

                # Adicionar os dados extraídos às respectivas listas
                games_ids.append(game_id)
                mandantes.append(mandante)
                visitantes.append(visitante)
                odds_mandantes.append(float(odd_mandante.replace(',', '.')))
                odds_empates.append(float(odd_empate.replace(',', '.')))
                odds_visitantes.append(float(odd_visitante.replace(',', '.')))
            except NoSuchElementException:
                # Se o elemento do jogo não for encontrado, interrompe o loop interno
                break
    
    # Criar um DataFrame pandas com os dados extraídos
    df = pd.DataFrame({
        'game_id': games_ids,
        'mandante': mandantes,
        'visitante': visitantes,
        'odd_mandante': odds_mandantes,
        'odd_empate': odds_empates,
        'odd_visitante': odds_visitantes
    })
    
    # Fechar o navegador
    browser.quit()
    
    # Retornar o DataFrame contendo os dados dos jogos e suas odds
    return df


In [22]:
# Chama a função get_betway_odds que acessa o site da Betway, extrai as odds e retorna um DataFrame pandas
betway_odds_df = get_betway_odds()

In [23]:
betway_odds_df

,game_id,mandante,visitante,odd_mandante,odd_empate,odd_visitante
0,cruzeiro-cuiaba,cruzeiro,cuiaba,1.61,3.50,6.00
1,athletico_pr-criciuma,athletico_pr,criciuma,1.45,4.20,7.00
2,flamengo-gremio,flamengo,gremio,1.53,4.00,6.00
3,internacional-sao_paulo,internacional,sao_paulo,2.80,3.00,2.62
4,bahia-fortaleza,bahia,fortaleza,1.95,3.40,3.80
5,palmeiras-vasco_da_gama,palmeiras,vasco_da_gama,1.40,4.50,7.50
6,bragantino-juventude,bragantino,juventude,1.57,3.80,5.75
7,atletico_goianiense-fluminense,fluminense,atletico_goianiense,1.70,3.80,4.50
8,athletico_pr-flamengo,athletico_pr,flamengo,2.80,3.10,2.50
9,corinthians-sao_paulo,corinthians,sao_paulo,3.10,3.00,2.40
